In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
# model_dir = 'yolox'
# model_name = 'yolox_x_8x8_300e'
# cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
# cfg = Config.fromfile('./configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py')
# cfg = Config.fromfile('./configs/faster_rcnn/yolox_x_8x8_300e_coco.py')
# cfg = Config.fromfile('./configs/'+model_dir+'/'+model_name+'_coco.py')

In [3]:
cfg = Config.fromfile('./configs/efficientnet/retinanet_effb3_fpn_crop896_8x4_1x_coco.py')

In [4]:
root='../../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.model.bbox_head.num_classes=10

In [5]:
cfg.data.train.ann_file = root + 'train.json' # train json 정보
# cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/retinanet_effb3_fpn_crop896_8x4_1x_trash'
# cfg.work_dir = './work_dirs/'+model_name+'_trash'
# cfg.model.roi_head.bbox_head.nu

# cfg.model.roi_head.bbox_head[0].num_classes = 10
# cfg.model.roi_head.bbox_head[1].num_classes = 10
# cfg.model.roi_head.bbox_head[2].num_classes = 10
# cfg.model.roi_head.mask_head.num_classes=10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

In [6]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.10s)
creating index...
index created!


In [7]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [8]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-01-11 16:14:55,988 - mmcv - INFO - initialize EfficientNet with init_cfg {'type': 'Pretrained', 'prefix': 'backbone', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b3_3rdparty_8xb32-aa_in1k_20220119-5b4887a0.pth'}
2024-01-11 16:14:55,989 - mmcv - INFO - load backbone in model from: https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b3_3rdparty_8xb32-aa_in1k_20220119-5b4887a0.pth
2024-01-11 16:14:56,087 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: layers.6.bn.running_var, layers.6.bn.running_mean, layers.6.bn.weight, layers.6.bn.bias, layers.6.conv.weight

2024-01-11 16:14:56,111 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}


load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b3_3rdparty_8xb32-aa_in1k_20220119-5b4887a0.pth


2024-01-11 16:14:56,201 - mmcv - INFO - 
backbone.layers.0.conv.weight - torch.Size([40, 3, 3, 3]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b3_3rdparty_8xb32-aa_in1k_20220119-5b4887a0.pth 
 
2024-01-11 16:14:56,202 - mmcv - INFO - 
backbone.layers.0.bn.weight - torch.Size([40]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b3_3rdparty_8xb32-aa_in1k_20220119-5b4887a0.pth 
 
2024-01-11 16:14:56,203 - mmcv - INFO - 
backbone.layers.0.bn.bias - torch.Size([40]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b3_3rdparty_8xb32-aa_in1k_20220119-5b4887a0.pth 
 
2024-01-11 16:14:56,204 - mmcv - INFO - 
backbone.layers.1.0.depthwise_conv.conv.weight - torch.Size([40, 1, 3, 3]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b3_3rdparty_8xb32-aa_in1k_20220119-5b4

In [9]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2024-01-11 16:14:58,382 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
/opt/conda/lib/python3.10/site-packages/mmcv/runner/optimizer/default_constructor.py:186: UserWarning: bbox_head.cls_convs.1.0.conv is duplicate. It is skipped since bypass_duplicate=True
  warnings.warn(f'{prefix} is duplicate. It is skipped since '
/opt/conda/lib/python3.10/site-packages/mmcv/runner/optimizer/default_constructor.py:186: UserWarning: bbox_head.cls_convs.1.1.conv is duplicate. It is skipped since bypass_duplicate=True
  warnings.warn(f'{prefix} is duplicate. It is skipped since '
/opt/conda/lib/python3.10/site-packages/mmcv/runner/optimizer/default_constructor.py:186: UserWarning: bbox_head.cls_convs.1.2.conv is duplicate. It is skipped since bypass_duplicate=True
  warnings.warn(f'{prefix} is duplicate. It is skipped since '
/opt/conda/lib/python3.10/site-packages/mmcv/runner/optimizer/default_constructor.py:186: UserWarning: bbox_head.cls_convs.1.3.conv is duplicate. I